In [608]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Reading the URL

In [609]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
#print(soup.prettify())
article=soup.find('table')



Reading the table with postcode and borough details into a dataframe

In [610]:
data = pd.DataFrame()
n_columns = 0
n_rows=0
column_names = []

table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
#    print(row) 
    td_tags = row.find_all('td') 
    if len(td_tags) > 0:
        n_rows+=1
    if n_columns == 0:
        # Set the number of columns for our table
        n_columns = len(td_tags)
        
    th_tags = row.find_all('th') 
    if len(th_tags) > 0 and len(column_names) == 0:
        for th in th_tags:
            column_names.append(th.get_text())
if len(column_names) > 0 and len(column_names) != n_columns:
        raise Exception("Column titles do not match the number of columns")       
    
columns = column_names if len(column_names) > 0 else range(0,n_columns)

df = pd.DataFrame(columns = columns,
                    index= range(0,n_rows))
row_marker = 0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        df.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    if len(columns) > 0:
        row_marker += 1
                    
     

Correctng the header column and removing Borough that are "Not Assigned"

In [611]:
df.rename(columns={'Neighbourhood\n':'Neighbourhood'},inplace=True)
newdf= df[df.Borough!='Not assigned']
newdf.reset_index(drop=True,inplace=True)
newdf2=newdf.copy()

createing a function to clean up "Neighbourhood" column that has extra character \n

In [612]:
def myreplacer(x):
    return x.replace('\n','')

In [613]:
newdf2['Neighbourhood2'] = newdf2.apply(lambda row: myreplacer(row['Neighbourhood']), axis=1)
newdf2.drop('Neighbourhood',inplace=True,axis=1)
newdf3=newdf2.rename(columns={'Neighbourhood2':'Neighbourhood'})

Implementing the logic If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.


In [614]:
for i, row in newdf3.iterrows():
    ifor_val = row
   # print(i,newdf3.iloc[i]['Neighbourhood'],)
    if (newdf3.iloc[i]['Neighbourhood'] == 'Not assigned'):
  #      print('yes it is',newdf3.iloc[i]['Borough'])
   #     ifor_val = ifor_val.Borough
        newdf3.at[i,'Neighbourhood'] = newdf3.at[i,'Borough']
        #newdf3.set_value(i,'ifor',ifor_val)

Implementing logic to group neighborhood that exist in one postal code area. 

In [615]:
newdf4=newdf3.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda Neighbourhood: ','.join(Neighbourhood)).reset_index()

In [616]:
newdf4.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [617]:
newdf4.shape

(103, 3)

Second Assignment

In [618]:
gdata=pd.read_csv('http://cocl.us/Geospatial_data')



In [619]:
gdata.rename(columns={'Postal Code':'Postcode'},inplace=True)

In [620]:
newdf5=newdf4.merge(gdata,on='Postcode',how='left')

In [621]:
newdf5.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Third Assignment 

In [622]:
address = 'Toronto, CN'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6425637, -79.3870871832.


In [623]:
str=('Downtown Toronto','West Toronto','East Toronto','Central Toronto')
Toronto_Borough=newdf5[newdf5.Borough.isin (str)]

In [643]:
CLIENT_ID = 'DUMMY' # your Foursquare ID
CLIENT_SECRET = 'DUMMY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DUMMY
CLIENT_SECRET:DUMMY


In [625]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [626]:
toronto_venues = getNearbyVenues(names=Toronto_Borough['Borough'],
                                   latitudes=Toronto_Borough['Latitude'],
                                   longitudes=Toronto_Borough['Longitude']
                                  )


East Toronto
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
West Toronto
East Toronto


In [627]:
print(toronto_venues.shape)
toronto_venues.head()

(1704, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,East Toronto,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,East Toronto,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,East Toronto,43.676357,-79.293031,Guru Raghavendra Ji,43.680187,-79.292337,Astrologer
3,East Toronto,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,East Toronto,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [628]:
toronto_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,112,112,112,112,112,112
Downtown Toronto,1288,1288,1288,1288,1288,1288
East Toronto,125,125,125,125,125,125
West Toronto,179,179,179,179,179,179


In [629]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


In [630]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,East Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [631]:
toronto_onehot.shape

(1704, 241)

In [632]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017857,0.000000,...,0.008929,0.000000,0.008929,0.000000,0.008929,0.000000,0.000000,0.000000,0.000000,0.008929
1,Downtown Toronto,0.000776,0.000776,0.000776,0.000776,0.001553,0.002329,0.001553,0.015528,0.001553,...,0.000776,0.002329,0.012422,0.002329,0.003882,0.006211,0.000000,0.000776,0.000776,0.002329
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024000,0.000000,...,0.008000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024000
3,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005587,...,0.000000,0.000000,0.011173,0.000000,0.011173,0.005587,0.005587,0.000000,0.000000,0.005587


In [633]:
toronto_grouped.shape

(4, 241)

In [634]:
num_top_venues = 5

for hood in toronto_grouped['Borough']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.08
1  Sandwich Place  0.06
2     Pizza Place  0.05
3            Park  0.05
4    Dessert Shop  0.04


----Downtown Toronto----
                venue  freq
0         Coffee Shop  0.09
1                Café  0.06
2          Restaurant  0.03
3  Italian Restaurant  0.03
4              Bakery  0.03


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.06
2  Italian Restaurant  0.05
3                Café  0.04
4      Ice Cream Shop  0.04


----West Toronto----
         venue  freq
0          Bar  0.08
1         Café  0.06
2  Coffee Shop  0.05
3   Restaurant  0.03
4  Pizza Place  0.03




In [635]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [636]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Sushi Restaurant,Clothing Store,Dessert Shop,Pub,Gym
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Bakery,Italian Restaurant,Japanese Restaurant,Bar,Seafood Restaurant,Steakhouse
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Park,Ice Cream Shop,Café,Brewery,Bakery,Furniture / Home Store,Pizza Place
3,West Toronto,Bar,Café,Coffee Shop,Italian Restaurant,Pizza Place,Bakery,Restaurant,Breakfast Spot,Music Venue,Gift Shop


In [637]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 2, 1], dtype=int32)

In [638]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_Borough

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.reset_index(drop=True,inplace=True)

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Park,Ice Cream Shop,Café,Brewery,Bakery,Furniture / Home Store,Pizza Place
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Park,Ice Cream Shop,Café,Brewery,Bakery,Furniture / Home Store,Pizza Place
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Park,Ice Cream Shop,Café,Brewery,Bakery,Furniture / Home Store,Pizza Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Park,Ice Cream Shop,Café,Brewery,Bakery,Furniture / Home Store,Pizza Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Coffee Shop,Sandwich Place,Pizza Place,Park,Café,Sushi Restaurant,Clothing Store,Dessert Shop,Pub,Gym


In [639]:

toronto_merged['Borough']=toronto_merged['Borough'].to_string()
type(toronto_merged[['Borough']].to_string())

toronto_merged['Borough']=toronto_merged['Borough'].str.encode('utf-8')

In [642]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(poi, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
